# "Silver Schema : Date Dimension"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
    .appName("Employee") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.jars", "/Drivers/SQL_Sever/jdbc/sqljdbc42.jar")\
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
spark.sql("show schemas;").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
|    sales|
|   silver|
+---------+



In [4]:
spark.sql("use bronze")

DataFrame[]

In [5]:
spark.sql("show tables;").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|   bronze|dimcustomer|      false|
|   bronze|    dimdate|      false|
|   bronze|dimemployee|      false|
|   bronze| dimproduct|      false|
|   bronze|  factsales|      false|
+---------+-----------+-----------+



In [6]:
bronze_Date = spark.sql("select * from bronze.dimdate")
bronze_Date.columns

['DateKey', 'Date', 'Year', 'Month', 'Day', 'Quarter']

In [7]:
bronze_Date.select("year").distinct().show()

+----+
|year|
+----+
|2012|
|2014|
|2013|
|2011|
+----+



In [8]:
bronze_Date.write.format("hive").mode("overwrite").saveAsTable("silver.dimdate")

In [9]:
spark.sql("use silver")

DataFrame[]

In [10]:
spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|   silver|dimcustomer|      false|
|   silver|    dimdate|      false|
+---------+-----------+-----------+



0: jdbc:hive2://localhost:10000/default> show schemas;
+----------------+
| database_name  |
+----------------+
| bronze         |
| default        |
| sales          |
| silver         |
+----------------+
4 rows selected (0.22 seconds)
0: jdbc:hive2://localhost:10000/default> use silver;
No rows affected (0.027 seconds)
0: jdbc:hive2://localhost:10000/default> select * from dimdate
. . . . . . . . . . . . . . . . . . . .> limit 5;
+------------------------+---------------+---------------+----------------+--------------+------------------+  
|    dimdate.datekey     | dimdate.date  | dimdate.year  | dimdate.month  | dimdate.day  | dimdate.quarter  |  
+------------------------+---------------+---------------+----------------+--------------+------------------+  
| 2011-10-10 00:00:00.0  | 2011-10-10    | 2011          | 10             | 10           | 4                |  
| 2011-12-04 00:00:00.0  | 2011-12-04    | 2011          | 12             | 04           | 4                |  
| 2012-01-22 00:00:00.0  | 2012-01-22    | 2012          | 01             | 22           | 1                |  
| 2012-07-11 00:00:00.0  | 2012-07-11    | 2012          | 07             | 11           | 3                |  
| 2012-10-20 00:00:00.0  | 2012-10-20    | 2012          | 10             | 20           | 4                |  
+------------------------+---------------+---------------+----------------+--------------+------------------+  
5 rows selected (0.379 seconds)